# Implementação de uma Rede Neural com Tensorflow

No exemplo a seguir, veremos em ação a uma Rede Neural Convolucional (CNN) em um problema de classificação de imagens. Queremos mostrar o processo de construção de uma rede CNN: quais são as etapas para executar e que raciocínio precisa ser feito para executar um dimensionamento adequado de toda a rede e, claro, como implementá-lo com o TensorFlow.

Carreguando e preparamdp os dados do MNIST:

In [1]:
import tensorflow as tf
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/Users/ramonfsilva/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Defina todos os parâmetros da CNN:

In [2]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

Entrada de dados MNIST (cada forma é de 28x28 pixels de matriz):

In [3]:
n_input = 784

O total de 10 classes do MNIST (0-9 dígitos)

In [4]:
n_classes = 10

Para reduzir o encaixe, aplicamos a técnica de dropout. Este termo refere-se ao abandono de unidades (oculto, entrada e saída) em uma rede neural. Decidir quais neurônios eliminar é aleatório; Uma maneira é aplicar uma probabilidade, como veremos em nosso código. Por esse motivo, definimos o seguinte parâmetro (a ser ajustado):

In [5]:
dropout = 0.75 

Defina os espaços reservados para o gráfico de entrada. O espaço reservado x contém a entrada de dados MNIST (exatamente 728 pixels):

In [6]:
x = tf.placeholder(tf.float32, [None, n_input])

Em seguida, mudamos a forma das imagens de entrada 4D para um tensor, usando o operador de reformulação TensorFlow:

In [7]:
_X = tf.reshape(x, shape=[-1, 28, 28, 1])

A segunda e terceira dimensões correspondem à largura e altura da imagem, enquanto a última dimensão é o número total de canais de cor (no nosso caso 1).

Assim, podemos exibir nossa imagem de entrada como um tensor bidimensional, de tamanho 28x28:
    
![title](img/digit.jpg)

In [8]:
y = tf.placeholder(tf.float32, [None, n_classes])

### Primeira camada convolucional
Cada neurônio da camada oculta é conectado a um pequeno subconjunto do tensor de entrada de dimensão 5x5. Isso implica que a camada oculta terá um tamanho 24x24. Também definimos e inicializamos os tensores de pesos compartilhados e viés compartilhado:

In [9]:
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32])) 
bc1 = tf.Variable(tf.random_normal([32]))

Lembre-se de que, para reconhecer uma imagem, precisamos de mais do que um mapa de recursos. O número é apenas o número de mapas de recursos que estamos considerando para essa primeira camada. No nosso caso, a camada convolucional é composta por 32 mapas de características.

O próximo passo é a construção da primeira camada de convolução, conv1:

In [11]:
def conv2d(img, w, b):
  return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w,strides=[1, 1, 1, 1],padding='SAME'),b))

conv1 = conv2d(_X,wc1,bc1)

Uma maneira de representar a camada convolucional, ou seja, conv1, é a seguinte:


![title](img/first_hidden_layer.jpg)

Após a operação de convolução, impomos a etapa de agrupamento que simplifica as informações de saída da camada convolucional criada anteriormente.

Em nosso exemplo, vamos considerar uma região 2x2 da camada de convolução e resumiremos as informações em cada ponto da camada de pooling.

In [14]:
def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

conv1 = max_pool(conv1, k=2)

A figura a seguir mostra as camadas CNNs após a operação de agrupamento e convolução:

![title](img/max_pooling.jpg)

A última operação é reduzir o overfitting aplicando os operadores tf.nn.dropout TensorFlow na camada convolucional. Para fazer isso, criamos um marcador de posição para a probabilidade (keep_prob) de que a saída de um neurônio é mantida durante o abandono

In [15]:
keep_prob = tf. placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1,keep_prob)

### Segunda camada convolucional
Para a segunda camada oculta, devemos aplicar as mesmas operações que a primeira camada, e assim definimos e inicializamos os tensores de pesos compartilhados e viés compartilhado:

In [16]:
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

Como você pode notar, esta segunda camada oculta terá 64 recursos para uma janela 5x5, enquanto o número de camadas de entrada será dado a partir da primeira camada obtida pela convolução. Em seguida, aplicamos uma segunda camada ao tensor convolucional conv1, mas desta vez aplicamos 64 conjuntos de filtros 5x5 cada para as 32 camadas conv1:

In [17]:
conv2 = conv2d(conv1,wc2,bc2)

Isso nos dá 64 arrays 14x14 que reduzimos com o pool máximo para 64 arrays 7x7:

In [18]:
conv2 = max_pool(conv2, k=2)

Finalmente, novamente usamos a operação de dropout:

In [19]:
conv2 = tf.nn.dropout(conv2, keep_prob)